### Simple ReAct Agent from Scratch

In [10]:
import os
import re
import google.generativeai as genai
from google.generativeai import GenerationConfig
from dotenv import load_dotenv


#####  1. Initialize Gemini Client

In [11]:
# Load environment variables from .env file
_ = load_dotenv()

api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("API_KEY not set.")

# Configure the Gemini client with the API key
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content("Explain how AI works")
print(response.text)

AI works by mimicking human cognitive functions, like learning, problem-solving, and decision-making, through algorithms and statistical models.  There's no single "how" because AI encompasses a broad range of techniques, but here's a breakdown of key aspects:

**1. Data is King:**  AI systems learn from data.  The more relevant and high-quality data they're trained on, the better they perform. This data can be structured (like tables in a database) or unstructured (like images, text, or audio).

**2. Algorithms: The Recipe for Intelligence:** Algorithms are sets of rules and instructions that tell the computer how to process the data.  Different algorithms are suited for different tasks. Some key types include:

* **Machine Learning (ML):** This is a core component of many AI systems.  ML algorithms allow the system to learn patterns and make predictions from data without explicit programming.  Instead of being explicitly programmed, they learn from examples.  There are several types 

##### 2. Available Actions for the Agent

In [12]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name == "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name == "Border Collie":
        return "a Border Collie's average weight is 37 lbs"
    elif name == "Toy Poodle":
        return "a Toy Poodle's average weight is 7 lbs"
    else:
        return "An average dog weighs 50 lbs"

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}


##### 3. Agent Class

In [13]:
class Agent:
    def __init__(self, system=""):
        self.messages = []
        if system:
            self.messages.append({
                "role": "user",
                "parts": [{"text": system}]
            })

    def __call__(self, message):
        self.messages.append({
            "role": "user",
            "parts": [{"text": message}]
        })
        result = self.execute()
        self.messages.append({
            "role": "model",
            "parts": [{"text": result}]
        })
        return result

    def execute(self):
        generation_config = GenerationConfig(
            temperature=0,
            max_output_tokens=2048,
            top_p=1,
            top_k=1
        )

        try:
            response = model.generate_content(
                contents=self.messages,
                generation_config=generation_config
            )
            return response.text
        except Exception as e:
            return f"Error: {str(e)}"


#####  4. Prompt for the Agent

In [14]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()



##### 5. Query Function with Action Handling Loop

In [15]:
# Action regex pattern
action_re = re.compile(r'^Action: (\w+): (.*)$')


def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return


##### 6. Testing

In [17]:
# Initialize the agent
abot = Agent(prompt)

result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should use the `average_dog_weight` action to find the average weight of a toy poodle.

Action: average_dog_weight: Toy Poodle
PAUSE



In [18]:
result = average_dog_weight("Toy Poodle")
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

'Answer: A toy poodle weighs approximately 7 lbs.\n'

In [10]:
# Using query function

question = "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight?"
query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together.  I'll start with the Border Collie.
Action: average_dog_weight: Border Collie
PAUSE

 -- running average_dog_weight Border Collie
Observation: a Border Collie's average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE

 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I have the average weights of both breeds, so I can add them together to get the combined weight.
Action: calculate: 37 + 20
PAUSE

 -- running calculate 37 + 20
Observation: 57
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.

